In [3]:
!pip install -U kaleido

ERROR: Operation cancelled by user


In [4]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/Only negative Reviews.csv', encoding = " **ISO**-8859-1 " )
data


,id,text
0,1,"I have return the products, but still not yet ..."
1,2,I ordered some products on 20th January 2023. ...
2,3,Very bad quality products specially veggies an...
3,4,"If we complain anything about things delivery,..."
4,5,Been a regular user of BB. But now there are m...
...,...,...
2043,2044,Hi folks.... Worst experience with bigbasket t...
2044,2045,"Iam a regular customer of bb daily, some thing..."
2045,2046,I got mangoes in over ripe condition. Requeste...
2046,2047,But I have not received a call back.


In [5]:
from collections import Counter
#from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
#from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
pat3 = r'(\\x(.){2})'
combined_pat = r'|'.join((pat1, pat2, pat3))
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()
testing = data['text']
test_result = []
for t in testing:
    test_result.append(tweet_cleaner(t))
test_result[0]

<ipython-input-6-99c4ffdbcb66>:9: MarkupResemblesLocatorWarning:

The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.

<ipython-input-6-99c4ffdbcb66>:9: MarkupResemblesLocatorWarning:

The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.



'i have return the products but still not yet received last since th jan rupee cheated not given back amount'

In [7]:
from pandas import DataFrame
df = DataFrame (test_result,columns=['Clean Text'])
print (df)

                                             Clean Text
0     i have return the products but still not yet r...
1     i ordered some products on th january out of t...
2     very bad quality products specially veggies an...
3     if we complain anything about things delivery ...
4     been a regular user of bb but now there are ma...
...                                                 ...
2043  hi folks worst experience with bigbasket today...
2044  iam a regular customer of bb daily some thing ...
2045  i got mangoes in over ripe condition requested...
2046                but i have not received a call back
2047                                           nonsense

[2048 rows x 1 columns]


In [8]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = test_result
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['have', 'return', 'the', 'products', 'but', 'still', 'not', 'yet', 'received', 'last', 'since', 'th', 'jan', 'rupee', 'cheated', 'not', 'given', 'back', 'amount']


In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [10]:
# NLTK Stop words
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['bigbasket', 'blinkit', 'big', 'basket', 'also','bb','big basket'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
#!pip install spacy
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'ADV'])
print(data_lemmatized[:1])

[['product', 'still', 'yet', 'last', 'amount']]


In [12]:
data_lemmatized

[['product', 'still', 'yet', 'last', 'amount'],
 ['product',
  'item',
  'request',
  'refund',
  'back',
  'time',
  'refund',
  'useless',
  'still',
  'money',
  'already',
  'day',
  'multiple',
  'mail',
  'updation',
  'clear',
  'reckless',
  'customer',
  'issue',
  'end',
  'well',
  'money'],
 ['bad',
  'quality',
  'product',
  'specially',
  'fruit',
  'bad',
  'ask',
  'revert',
  'back',
  'sometimes',
  'issue'],
 ['thing',
  'delivery',
  'sort',
  'problem',
  'instead',
  'milk',
  'veggie',
  'fruit',
  'properly',
  'customer',
  'care',
  'namesake',
  'thast',
  'service',
  'daily'],
 ['regular',
  'user',
  'major',
  'issue',
  'payment',
  'payment',
  'straight',
  'payment',
  'successful',
  'window',
  'cancel',
  'able',
  'delivery',
  'way'],
 ['bad',
  'pathetic',
  'experience',
  'executive',
  'soft',
  'skill',
  'fool',
  'download',
  'subscription'],
 ['unnecessarily',
  'amount',
  'delivery',
  'charge',
  'charge',
  'day',
  'delivery',
  'c

In [13]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [14]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics= 13,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=1000,
                                           per_word_topics=True)

In [15]:
#I removed the comments to make the bloc shorter
def format_topics_sentences(corpus, texts, ldamodel=None):
    data = []

    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        row = sorted(row, key=lambda x: x[1], reverse=True)

        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                data.append([int(topic_num), round(prop_topic, 4), topic_keywords])
            else:
                break

    sent_topics_df = (
        pd.DataFrame(data, columns=[
            'Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords'])
                .assign(Text= texts) # or use Text= pd.Series(text)
    )

    return sent_topics_df

In [16]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_lemmatized)
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(13)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2,0.3751,"item, time, product, problem, delivery, really...","[product, still, yet, last, amount]"
1,1,2,0.3203,"item, time, product, problem, delivery, really...","[product, item, request, refund, back, time, r..."
2,2,10,0.6513,"good, bad, fruit, com, never, quality, people,...","[bad, quality, product, specially, fruit, bad,..."
3,3,4,0.3173,"issue, instead, veggie, replacement, cancellat...","[thing, delivery, sort, problem, instead, milk..."
4,4,3,0.8578,"service, customer, order, time, bad, even, day...","[regular, user, major, issue, payment, payment..."
5,5,11,0.3077,"account, review, mobile, health, card, rude, c...","[bad, pathetic, experience, executive, soft, s..."
6,6,2,0.9423,"item, time, product, problem, delivery, really...","[unnecessarily, amount, delivery, charge, char..."
7,7,3,0.8462,"service, customer, order, time, bad, even, day...","[pathetic, service, pathetic, customer, support]"
8,8,3,0.5328,"service, customer, order, time, bad, even, day...","[delivery, slot, fancy, message, delivery, cus..."
9,9,8,0.5165,"website, price, thing, shopping, second, groce...","[delivery, man, raju, deliver, food, door, fro..."


In [17]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 90

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet,
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)],
                                            axis=0)

# Reset Index
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(20)

sent_topics_sorteddf_mallet.to_csv("Negative 13_25_April_24.csv")

In [18]:
from gensim.matutils import hellinger
from scipy import spatial as scs
from scipy.cluster import hierarchy as sch
from scipy.spatial.distance import pdist, squareform


# get topic distributions
topic_dist = lda_model.state.get_lambda()

# get topic terms
num_words = 15
topic_terms = [{w for (w, _) in lda_model.show_topic(topic, topn=num_words)} for topic in range(topic_dist.shape[0])]

# no. of terms to display in annotation
n_ann_terms = 15

# use hellinger distance metric in dendrogram
def js_dist(X):
    return pdist(X, lambda u, v: hellinger(u, v))

# define method for distance calculation in clusters
linkagefun=lambda x: sch.linkage(x, 'single')

# calculate text annotations
def text_annotation(topic_dist, topic_terms, n_ann_terms, linkagefun):
    # get dendrogram hierarchy data
    linkagefun = lambda x: sch.linkage(x, 'single')
    d = js_dist(topic_dist)
    Z = linkagefun(d)
    P = sch.dendrogram(Z, orientation="bottom", no_plot=True)

    # store topic no.(leaves) corresponding to the x-ticks in dendrogram
    x_ticks = np.arange(5, len(P['leaves']) * 10 + 5, 10)
    x_topic = dict(zip(P['leaves'], x_ticks))

    # store {topic no.:topic terms}
    topic_vals = dict()
    for key, val in x_topic.items():
        topic_vals[val] = (topic_terms[key], topic_terms[key])

    text_annotations = []
    # loop through every trace (scatter plot) in dendrogram
    for trace in P['icoord']:
        fst_topic = topic_vals[trace[0]]
        scnd_topic = topic_vals[trace[2]]

        # annotation for two ends of current trace
        pos_tokens_t1 = list(fst_topic[0])[:min(len(fst_topic[0]), n_ann_terms)]
        neg_tokens_t1 = list(fst_topic[1])[:min(len(fst_topic[1]), n_ann_terms)]

        pos_tokens_t4 = list(scnd_topic[0])[:min(len(scnd_topic[0]), n_ann_terms)]
        neg_tokens_t4 = list(scnd_topic[1])[:min(len(scnd_topic[1]), n_ann_terms)]

        t1 = "<br>".join((": ".join(("+++", str(pos_tokens_t1))), ": ".join(("---", str(neg_tokens_t1)))))
        t2 = t3 = ()
        t4 = "<br>".join((": ".join(("+++", str(pos_tokens_t4))), ": ".join(("---", str(neg_tokens_t4)))))

        # show topic terms in leaves
        if trace[0] in x_ticks:
            t1 = str(list(topic_vals[trace[0]][0])[:n_ann_terms])
        if trace[2] in x_ticks:
            t4 = str(list(topic_vals[trace[2]][0])[:n_ann_terms])

        text_annotations.append([t1, t2, t3, t4])

        # calculate intersecting/diff for upper level
        intersecting = fst_topic[0] & scnd_topic[0]
        different = fst_topic[0].symmetric_difference(scnd_topic[0])

        center = (trace[0] + trace[2]) / 2
        topic_vals[center] = (intersecting, different)

        # remove trace value after it is annotated
        topic_vals.pop(trace[0], None)
        topic_vals.pop(trace[2], None)

    return text_annotations

In [19]:
import plotly.figure_factory as ff
import plotly.offline as py
# get text annotations
annotation = text_annotation(topic_dist, topic_terms, n_ann_terms, linkagefun)

# Plot dendrogram
names = ['Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic 5','Topic 6','Topic 7','Topic 8','Topic 9','Topic 10','Topic 11', 'Topic 12','Topic 13']
dendro = ff.create_dendrogram(topic_dist, distfun=js_dist, linkagefun=linkagefun, hovertext=annotation, orientation="right", labels=names)
dendro['layout'].update({'width': 1200, 'height': 600})
py.iplot(dendro)
file='/content/images/Dendrogram_Neg.png'
dendro.write_image(file)
